### DATA COLLECTING

In [5]:
import re
import pandas as pd
import json

from pythainlp.tokenize import THAI2FIT_TOKENIZER

# message converter
from project_module.message_data_converter import MessageDataConverter


In [8]:
json_file = open('./raw_data/somchai_message_1.json')
json_obj = json.load(json_file)
message_list = []

for message in json_obj['messages']:
    try:
        message_content = message['content'].encode('latin_1').decode('utf-8')
        message_list.append(message_content)
        
    except:
        continue

In [9]:
print(message_list)

['ออ รับทราบครับ', 'เดี๋ยวเดือนหน้า อ.รัฐชัย จะมาสลับสอนกับผม หรือบางครั้งอาจจะเป็น อ.มนตรี เราลองติดต่ออาจารย์ดูนะครับว่าชั่วโมงแล็บจะต้องช่วยอะไรบ้าง', 'เราตรวจเฉพาะงานที่ผมสั่งตอนทำแล็บนะครับ', 'ผมตรวจเองครับ', 'งานชิ้นนี้ผมต้องตรวจไหมครับ', 'โอเคครับ', 'ใช่ครับ', 'แสดงว่าสัปดาห์หน้าไม่มีแล็ปนะครับ', 'หยุดครับ', 'วันจันทร์ที่จะถึงนี้หยุดหรือเปล่าครับ', 'ออ ครับผม', 'โอเค ครับ ช่วงแรกผมจะให้เวลามากหน่อย คนที่ไม่มีพื้นฐานมาเลยจะได้ปรับตัวได้ เดี๋ยวอีกสัก 1-2 สัปดาห์ จะยากขึ้น', 'ในแล็ปยังทำกันได้อยู่ครับ', 'ส่วนใหญ่ทำกันได้ไหมครับ', '7 คน ครับ', 'ไม่ทราบว่ามาเข้าแล็ปกี่คนครับ', 'ช่วงนี้ อนุญาตให้ออนไลน์ได้ เพราะมีความเสี่ยงเรื่องโควิด19 พอสมควร', 'โอเคครับผม', 'ขอโทษด้วยครับ ผมลืมกำหนดคะแนน ค่า default เลยเป็น 100 ตอนนี้ผมปรับเป็น 10 แล้วครับ', 'ส่วนหลักการให้คะแนนงาน คือ ถ้าส่งครบหมดทุกข้อ ให้ 5 คะแนน (ถ้าไม่ครบ ก็หักไปตามสัดส่วนครับ เช่น ครั้งนี้มี 3 ข้อ ทำมา 2 ข้อ ก็หักออก 5/3 = 1.6 คะแนน เป็นต้น) แต่ละข้อถ้าทำถูกต้องสมบูรณ์ ก็ให้อีก 5 รวมเป็น 10 ครับ แต่ถ้ามีข้อไหน ทำผิด คือ ผิดไป

## Filter non contain thai character 

In [5]:
# finding constant values
text_bytes_code = 'ก'.encode('utf-8')
print(text_bytes_code)
print(len(text_bytes_code))

text_int_code = int.from_bytes(text_bytes_code, 'big')
print(text_int_code)

text_int_code += 62

text_bytes_code_last = text_int_code.to_bytes(len(text_bytes_code), 'big')
print(text_bytes_code_last)
text_last = text_bytes_code_last.decode('utf-8')

print(text_last)

b'\xe0\xb8\x81'
3
14727297
b'\xe0\xb8\xbf'
฿


In [6]:
# use constanst values to create map
thai_characters_offset = 14727297
thai_characters_bytes_len = 3
thai_characters = [ (index.to_bytes(thai_characters_bytes_len, 'big')).decode('utf-8') for index in range(thai_characters_offset, thai_characters_offset + 63) ]
print(thai_characters)

['ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช', 'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท', 'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ', 'ล', 'ฦ', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า', 'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', '\u0e3b', '\u0e3c', '\u0e3d', '\u0e3e', '฿']


In [7]:
# filtter messages
filter_func = lambda s: any(x in s for x in thai_characters)

message_list_filtered = [line for line in message_list if filter_func(line)]

print(message_list_filtered)

['ขอบคุณค่ะ', 'โอเคค่ะ', 'ให้อาจารย์เซ็นเรียบร้อยแล้วครับ', 'ได้ค่ะ', 'โอเคครับ ผมขอเอาไปให้อาจารย์เซ็นก่อนครับ', 'ทั้งสองคนเลย', 'มีค่ะ', 'ขอโทษครับ ผมลืมถามเรื่องลายเซ็นต้องมีของอาจารย์ที่ปรึกษากับอาจารย์รายวิชาเลยไหมครับ', 'เคค่ะ', 'ได้เลยครับ', 'ส่งให้พี่วันศุกร์ นะคะ', 'ขอบคุณค่ะ', 'โอเคครับ', 'กรอกใบสมัครด้วยนะ', 'เคค่ะ', 'คุยแล้วครับ', 'อาจารย์คุยกับน้องแล้วใช่ไหม', 'เวลาทำงาน', 'การทำงาน คือ ทำ 6 ชม. ต่อสัปดาห์นะคะ', 'มีตรวจงานบ้าง', 'งาน ก็มี ช่วยดูระหว่างน้องลงแลบ ค่ะ', 'น้องครับ', 'เคค่ะ', 'ยังเลยครับ', 'อาจารย์ เค้าทักมาคุยยัง', 'ขอบคุณค่ะ', 'รับทราบครับ', 'เดียว อ.สมชัย จะติดต่อเราผ่านทางเฟสบุ๊ค นะคะ', 'เคค่ะ', '087-6313043 ครับ', 'พี่ขอเบอร์โทรเราหน่อย', 'ได้ค่ะ', 'เวลาไม่ชนครับ ผมขอทราบลักษณะงานก่อนนะครับ', 'ได้ค่ะ', 'สักครู่นะครับ กำลังเช็คตารางอยู่ครับ', 'ถ้าได้ พี่ให้คุย กับอ.สมชัย ถามลักษณะงานก่อน', 'ช่วงดังกล่าว มีเรียนไหม', 'ได้ไหม', 'ทำงาน วัน จันทร์ เวลา 13.00 - 16.00 และวันพุธ เวลา 13.00 -16.00น.', 'พี่จะขอทาบทามเรา เป็นTA วิชา 241-151 หลักสูตร IA หน่อยได้ไหม', 

### EXPORT TO CSV

In [8]:
data = pd.DataFrame({'message': message_list_filtered})
data

,messages
0,ขอบคุณค่ะ
1,โอเคค่ะ
2,ให้อาจารย์เซ็นเรียบร้อยแล้วครับ
3,ได้ค่ะ
4,โอเคครับ ผมขอเอาไปให้อาจารย์เซ็นก่อนครับ
...,...
130,ทำงาน 3 ชม.ต่อสัปดาห์นะคะ
131,อันนี้ คือช่วงเวลาเรียน\nจันทร์ 11.00 - 11.50\...
132,ตอนนี้คุณสามารถส่งข้อความ โทรหากันและกัน และดู...
133,พี่ทาบทาม เราเป็น TA\nFUNDA MATHEMATICS FOR EN...


In [9]:
data.to_csv('message_csv.csv', encoding='utf-8-sig')

# Use Case

In [2]:
message_data_converter = MessageDataConverter()

In [4]:
message_data_converter.import_json(path='./raw_data/wimon_message_1.json')

['ขอบคุณค่ะ',
 'โอเคค่ะ',
 'ให้อาจารย์เซ็นเรียบร้อยแล้วครับ',
 'ได้ค่ะ',
 'โอเคครับ ผมขอเอาไปให้อาจารย์เซ็นก่อนครับ',
 'ทั้งสองคนเลย',
 'มีค่ะ',
 'ขอโทษครับ ผมลืมถามเรื่องลายเซ็นต้องมีของอาจารย์ที่ปรึกษากับอาจารย์รายวิชาเลยไหมครับ',
 'เคค่ะ',
 'ได้เลยครับ',
 'ส่งให้พี่วันศุกร์ นะคะ',
 'ขอบคุณค่ะ',
 'โอเคครับ',
 'กรอกใบสมัครด้วยนะ',
 'เคค่ะ',
 'คุยแล้วครับ',
 'อาจารย์คุยกับน้องแล้วใช่ไหม',
 'เวลาทำงาน',
 'การทำงาน คือ ทำ 6 ชม. ต่อสัปดาห์นะคะ',
 'มีตรวจงานบ้าง',
 'งาน ก็มี ช่วยดูระหว่างน้องลงแลบ ค่ะ',
 'น้องครับ',
 'เคค่ะ',
 'ยังเลยครับ',
 'อาจารย์ เค้าทักมาคุยยัง',
 'ขอบคุณค่ะ',
 'รับทราบครับ',
 'เดียว อ.สมชัย จะติดต่อเราผ่านทางเฟสบุ๊ค นะคะ',
 'เคค่ะ',
 '087-6313043 ครับ',
 'พี่ขอเบอร์โทรเราหน่อย',
 'ได้ค่ะ',
 'เวลาไม่ชนครับ ผมขอทราบลักษณะงานก่อนนะครับ',
 'ได้ค่ะ',
 'สักครู่นะครับ กำลังเช็คตารางอยู่ครับ',
 'ถ้าได้ พี่ให้คุย กับอ.สมชัย ถามลักษณะงานก่อน',
 'ช่วงดังกล่าว มีเรียนไหม',
 'ได้ไหม',
 'ทำงาน วัน จันทร์ เวลา 13.00 - 16.00 และวันพุธ เวลา 13.00 -16.00น.',
 'พี่จะขอทาบทามเรา เป็นTA 

In [6]:
message_data_converter.export_csv(path='./data/message_temp.csv')

In [3]:
# message_data_converter.merge_csv(csv1_path='./data/message_1.csv', csv2_path= './data/message_2.csv', des_path='./data/message_12.csv')

### Many JSON in folder to CSV

In [2]:
# init converter
message_data_converter = MessageDataConverter()

In [3]:
# mypath = './raw_data/'
# onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [7]:
# print(onlyfiles)
# for f in onlyfiles:
#     file_name = f.split('.')[0]
#     message_data_converter.import_json(path = f'./raw_data/{f}')
#     message_data_converter.export_csv(path = f'./data/{file_name}.csv')
#     message_data_converter.clear_message_list()


['somchai_message_1.json', 'wimon_message_1.json']


In [3]:
#completed many file convert func. in module
message_data_converter.convert_many(json_path= './raw_data/', csv_path= './data/')

### Merged Multiple CSV

In [3]:
message_data_converter.merge_many_csv(src_path='./test_data/', des_path='./data/')

### Count Words in Dataframe

In [18]:
data = pd.read_csv("./data/message_temp.csv")
data = data[['message', 'category']]

word_list = set()
for text in data['message']:
    
    text = text.lower().replace('\n', ' ').replace('\r', '').strip()
    text = re.findall(r"[\u0E00-\u0E7Fa-zA-Z']+", text)
    text = ' '.join(text)

    word_tokens = THAI2FIT_TOKENIZER.word_tokenize(text)
    filtered_sentence = set([w for w in word_tokens])

    word_list.update(filtered_sentence)

In [19]:
len(word_list)

319